# Aerospike Operations: Adding a Namespace while maintaining real time performance using AWS CloudFormation

*Last updated: September 27, 2021*

This tutorial on Aerospike Operations demonstrates how to add a [Namespace](https://docs.aerospike.com/docs/operations/configure/namespace/index.html) to an Aerospike Cluster while maintaining real time application performance. Using [AWS Quickstart](https://aws.amazon.com/marketplace/pp/prodview-ouzvxjdgfsa64) and [CloudFormation](https://aws.amazon.com/cloudformation/) facilitates infrastructure creation. The notebook will also help a new Aerospike administrator learn cluster administration basics.

This [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html) requires the Aerospike Database running locally. To create a Docker container that satisfies the requirements and holds a copy of these notebooks, visit the [Aerospike Notebooks Repo](https://github.com/aerospike-examples/interactive-notebooks).

# Prerequisites

Prerequisites to get the most out of this tutorial are familiarity with:
* Basic system administration
* AWS:
   * [EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/concepts.html)
   * [IAM](https://docs.aws.amazon.com/IAM/latest/UserGuide/getting-started.html)
   * Quickstart
   * CloudFormation 

Basic understanding of the Aerospike Database is helpful.  

# Scenario for Namespace Addition
An application adds a new major feature, which requires the addition of a new in-memory namespace named ```newns```. During this process an original 3-node (m5.large) cluster is replaced with a new (m5.xlarge) cluster. This example assumes the use of Aerospike’s mesh network. In a real world situation, this would be an ideal time to resize existing namespaces, as needed.

# Namespace change process overview
The following is an overview of the steps to execute a namespace addition without downtime. Additional context and details follow the lists of steps. 

**Note:** A Namespace change can be made without down time on any cluster with replication factor of 2 or more, and on any cluster running Aerospike Database 3.13.0.1+. Since the default replication factor for a 1-node Aerospike database is 2, this process can be used with most single-node Aerospike databases. This means that one-node Aerospike databases can grow from pilot instance to production scale cluster with 100% data availability and no downtime.

1. Create new nodes
2. Create common Aerospike.conf additions
3. Increase migrate-fill-delay for cluster
4. For each new node to be added to the Aerospike Cluster (1-by-1):
   1. Update Aerospike.conf
   2. Restart
   3. Change the AWS Security Group to match that of the original cluster nodes
   4. For the first new node, use the tip command to add it to the cluster
   5. Confirm it joins the cluster
5. Request to Quiesce each original node in the Aerospike Cluster
6. Set the migrate-fill-delay to 0
7. Recluster the cluster  
8. Wait for migrations to complete
9. Take down quiesced nodes
10. Clear old nodes from the alumni list

Many configuration and information commands are run using the Aerospike tools and [asadm](https://docs.aerospike.com/docs/tools/asadm) and [asinfo](https://docs.aerospike.com/docs/tools/asinfo/index.html), as executed via CLI on a cluster node. Asinfo commands will be run through asadm, except when necessary to run locally on a node. Where instructive, the commands will be demonstrated in the tutorial environment’s built-in Aerospike Server node. The best practice is to use asadm for these commands, and even to run asinfo commands within asadm whenever possible. 

To perform the following tasks on a cluster node, ssh to it. If using AWS Quickstart using the CloudFormation template, ssh by way of the bastion host to the cluster node. Run asadm using the `-h` option to connect remotely to a cluster.  

# Ensure the database is running

In [ ]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike Database is running." || echo "**Aerospike Database is not"

# Details for adding a Namespace while maintaining realtime performance
Rather than incur downtime when changing Namespace configurations, the best practice is to allow Aerospike’s resilient clustering to maintain data and cluster uptime during such a change. 

The following steps describe the purpose and show results from each step of replacing the nodes of an AWS Quickstart 3-node cluster.

## Create new nodes 
When CloudFormation was used to stand up the original cluster, the best practice is to use CloudFormation to create 3 new nodes – one at a time – and add each to the original cluster's VPC. 


## Create common Aerospike.conf additions
Using identical [aerospike.conf](https://docs.aerospike.com/docs/cloud/kubernetes/operator/Aerospike-configuration-mapping.html) configurations across clusters is a goal for the smoothest execution. To achieve this, create a list of new seed nodes and a common namespace configuration that includes old and new namespaces.

The best source for this configuration is the file ```/etc/aerospike/aerospike.conf``` on one of the previously existing Aerospike cluster nodes. 


### Create the list of new seed nodes
Collect the IP addresses for internal communication among the newly created Aerospike nodes. These will make up the seed list for the cluster resulting from this process. Format it like the following example:
```
               mesh-seed-address-port 10.0.19.115 3002

               mesh-seed-address-port 10.0.154.23 3002

               mesh-seed-address-port 10.0.151.216 3002
```
Keep the resulting string containing all seed nodes available to copy/paste later in the process.


### Create the new namespace configuration 
Collect the existing namespace configuration from existing nodes:
```
namespace test {
        replication-factor 2
        memory-size 4G

        storage-engine memory
}
namespace bar {
replication-factor 2
        memory-size 4G

        storage-engine memory

        # To use file storage backing, comment out the line above and use the   
	# following lines instead.                                              
#       storage-engine device {                                                 
#               file /opt/aerospike/data/bar.dat                                
#               filesize 16G   
#       }                                                                       
}
```

Add the new namespace as follows:
```
namespace newns {
        replication-factor 2
        memory-size 4G
        storage-engine memory
}
```
Keep the resulting string containing all Namespaces available to copy/paste later in the process.


## Increase migrate-fill-delay for cluster
Prevent the Aerospike cluster from migrating to the new nodes as they are added by setting a migrate-fill-delay for the first steps in this process. A migrate-fill-delay of 1 hour (3600 seconds) will suffice for this scenario. **Migration** refers to the balancing of data partitions in the cluster to ensure uniform partition ownership.  This configuration will minimize data migration and therefore maintain highest availability to application traffic. 

The command is: ```asadm -e "asinfo -v 'set-config:context=service;migrate-fill-delay=3600'"```

```
> asadm -e "asinfo -v 'set-config:context=service;migrate-fill-delay=3600'"
~~~ asinfo -v 'set-config:context=service;migrate-fill-delay=3600' ~~~
ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
ok

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
ok

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
ok
```

Run this command on the 1-node cluster in this Jupyter environment:

In [1]:
!asadm -e "enable;asinfo -v 'set-config:context=service;migrate-fill-delay=3600'"

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf

~~~ enable ~~~

~~~ asinfo -v 'set-config:context=service;migrate-fill-delay=3600' ~~~
1d5354ebebac:3000 (172.17.0.2) returned:
ok



**Note:** This tutorial was written using tools version 2.0.1, which is installed using AWS Quickstart using CloudFormation. This notebook tutorial uses version 2.2. Commands and outputs are recognizable and comparable. 

## Do the following steps on each new node 1-by-1
New nodes could form their own cluster before joining the original cluster if created at once. This would complicate migration. As a result, do the following on each new node, one at a time.

From a Cloud Formation perspective, it is common to use the same as the original cluster for all of the following AWS settings:
* [EC2 Key Pair](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html)
* [VPC](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Subnets.html) ID
* List of [subnet](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Subnets.html) IDs
* Aerospike [Feature key file](https://docs.aerospike.com/docs/operations/configure/feature-key/) – Can also be replaced manually during the next step.
* [Security Group](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_SecurityGroups.html) ID for the cluster’s [Bastion](https://aws.amazon.com/blogs/security/controlling-network-access-to-ec2-instances-using-a-bastion-server/#:~:text=A%20bastion%20is%20a%20special,to%20your%20other%20EC2%20instances.&text=To%20define%20the%20source%20IPs,the%20instance's%20security%20group%20rules.).


### Update Aerospike.conf
Ssh to each new cluster node and replace both of the following:
* The list of seed nodes with the list created in a previous step
* Default Namespace configuration with the new Namespace configuration created in a previous step


### Restart the node
Since the configuration of the Aerospike node has changed, restart the Aerospike service, using the following command: ```sudo service aerospike restart```

```
> sudo service aerospike restart
Redirecting to /bin/systemctl restart aerospike.service
```

### Change the AWS Security Group to match that of the original cluster
To allow new nodes to connect to the original cluster, change the AWS security group to match that of the original cluster. It can be helpful to change the IAM role, as well, though the underlying privileges of the newly created IAM role should be identical.


### For the first new node, use the tip command to add it to the cluster.  
Because the new nodes are not on the original cluster nodes’ seed list, use the tip command to authorize the cluster to include it. Remaining nodes will connect via the configuration alignment with this first new node.
```asadm -e "asinfo -v 'tip:host=HOST;port=MESH-PORT'"```

```
$ asadm -e "asinfo -v 'tip:host=10.0.19.115;port=3002'"
Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/ec2-user/.aerospike/astools.conf, /etc/aerospike/astools.conf
ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
ok

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
ok

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
ok
```

### Confirm that it joins the cluster.

Starting asadm on a cluster node will indicate the size and IPs of the cluster. 

```
> asadm
Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/ec2-user/.aerospike/astools.conf, /etc/aerospike/astools.conf
Aerospike Interactive Shell, version 2.0.1

Found 4 nodes
Online:  10.0.19.115:3000, 10.0.30.148:3000, 10.0.48.227:3000, 10.0.47.149:3000
```

After confirming that a new node joins the cluster, proceed to the next new node.


## Quiesce each original node in the Aerospike Cluster
At this point in the process, the cluster will have over twice the capacity of the original cluster. The remaining steps remove the original nodes.

**Quiescing** a node makes it no longer the authoritative source of any data in a cluster. This configuration is requested, polled for status change, and takes effect upon reclustering. Once a node is quiesced, subsequent client connections to a quiesced node will: 
* Communicate new data ownership.
* Proxy the request through the quiesced node to the new owner. (Any edge cases?) 

For more information on [Quiescing a node, go here](https://docs.aerospike.com/docs/operations/manage/cluster_mng/quiescing_node/). Quiescing is a robust cluster mechanism that is central to these steps. 

### Ensure the cluster is stable
Because Aerospike [clustering](https://docs.aerospike.com/docs/architecture/clustering.html) is adaptive, administration has numerous steps with occasional delays while the cluster adapts, but there is little expectation of failure. When describing an Aerospike cluster, **stable** refers to whether all expected nodes are available, part of the cluster, and not migrating data. Aerospike clusters are highly performant, even during periods of cluster adaptation. 
 
Replace ```<Size>``` with the expected current number of cluster nodes and run on one cluster node. 
```asadm -e "asinfo -v 'cluster-stable:size=<Size>;ignore-migrations=no'"```

If there are no migrations in progress, each cluster node will return the [```cluster_key```](https://docs.aerospike.com/docs/reference/metrics/?show-removed=1#cluster_key).

```
> asadm -e "asinfo -v 'cluster-stable:size=4;ignore-migrations=no'"
ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
206FA51ADD40

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
206FA51ADD40

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
206FA51ADD40

ip-10-0-19-115.us-east-2.compute.internal:3000 (10.0.19.115) returned:
206FA51ADD40
```

Run this command locally, on the 1-node cluster in this Jupyter environment: 

In [2]:
!asadm -e "enable;asinfo -v 'cluster-stable:size=1;ignore-migrations=no'"

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf

~~~ enable ~~~

~~~ asinfo -v 'cluster-stable:size=1;ignore-migrations=no' ~~~
1d5354ebebac:3000 (172.17.0.2) returned:
186B770583C4



### Request quiesce on each of the old cluster nodes 
AWS Quickstart installs tools version 2.0.1. Using any 2.X version of the tools, an admin can ssh to each of the original nodes in the cluster and run the following command:
```> asinfo -v 'quiesce:'```

```
$ asinfo -v 'quiesce:'
Ok
```

Run this command locally:

In [3]:
!asinfo -v 'quiesce:'

ok


If using asadm 2.2+ and Aerospike 4.3.1+, one can quiesce a node from any node in the cluster. 

In asadm 2.2+, use the following command: 
```asadm -e "enable;asinfo -v 'quiesce:' with AAA.BBB.CCC.DDD"```
Replace ```AAA.BBB.CCC.DDD``` with the IP address of the cluster node to quiesce

Run this command locally:

In [4]:
!asadm -e "enable;asinfo -v 'quiesce:' with 172.17.0.2"

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf

~~~ enable ~~~

~~~ asinfo -v 'quiesce:' with 172.17.0.2 ~~~
1d5354ebebac:3000 (172.17.0.2) returned:
ok



Using asadm 2.3+, one can quiesce a node using the following command:
```asadm -e "enable;manage quiesce with AAA.BBB.CCC.DDD"```

```
~~~Quiesce Nodes~~~~
       Node|Response
ubuntu:3000|ok
Number of rows: 1
```

### Confirm quiesce configuration
Once all nodes have been requested to quiesce, make sure that all nodes in the cluster are aware of the status using the command: ```asadm -e 'show statistics like pending_quiesce'``` 

***Note:*** Each node will respond for each Namespace. 

When expanded, each node will have a vertical column listing its [```pending_quiesce```](https://docs.aerospike.com/docs/reference/metrics/index.html#pending_quiesce) status: true/false/--. 

An original node will look like this:
```
> asadm -e 'show statistics like pending_quiesce'

~~~~~~newns Namespace Statistics (2021-09-15 00:04:01 UTC)~~~~~~~
Node          |ip-10-0-30-148.us-east-2.compute.internal:3000
pending_quiesce|true
Number of rows: 2

~~~~~~newns Namespace Statistics (2021-09-15 00:04:01 UTC)~~~~~~
Node          |ip-10-0-30-148.us-east-2.compute.internal:3000
pending_quiesce|--                                                           
Number of rows: 2

~~~~~~test Namespace Statistics (2021-09-15 00:04:01 UTC)~~~~~~
Node          |ip-10-0-30-148.us-east-2.compute.internal:3000
pending_quiesce|true
Number of rows: 2
```

```--``` indicates that a node is not configured for a particular Namespace, and its status is neither true nor false.


A new node will look like this:
```
~~~~~~bar Namespace Statistics (2021-09-15 00:04:01 UTC)~~~~~~~
Node           |ip-10-0-151-216.us-east-2.compute.internal:3000
pending_quiesce|false
Number of rows: 2

~~~~~~newns Namespace Statistics (2021-09-15 00:04:01 UTC)~~~~~~
Node           |ip-10-0-151-216.us-east-2.compute.internal:3000
pending_quiesce|false                                                           
Number of rows: 2

~~~~~~test Namespace Statistics (2021-09-15 00:04:01 UTC)~~~~~~
Node           |ip-10-0-151-216.us-east-2.compute.internal:3000
pending_quiesce|false  
Number of rows: 2
```

### Set the migrate-fill-delay to 0
The cluster is now ready for efficient migration. Original nodes have been asked to quiesce but the quiesce has not taken effect, and new nodes are part of the cluster and have greater capacity than the original nodes. Remove the migrate-fill-delay by setting it to 0 using the command: 
```asadm -e "asinfo -v 'set-config:context=service;migrate-fill-delay=0'"```

```
> asadm -e "asinfo -v 'set-config:context=service;migrate-fill-delay=0'"
~~~ asinfo -v 'set-config:context=service;migrate-fill-delay=0' ~~~
ip-10-0-151-216.us-east-2.compute.internal:3000 (10.0.151.216) returned:
ok

ip-10-0-154-23.us-east-2.compute.internal:3000 (10.0.154.23) returned:
ok

ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
ok

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
ok

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
ok

ip-10-0-19-115.us-east-2.compute.internal:3000 (10.0.19.115) returned:
ok
```

Run the command locally:

In [5]:
!asadm -e "enable;asinfo -v 'set-config:context=service;migrate-fill-delay=0'"

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf

~~~ enable ~~~

~~~ asinfo -v 'set-config:context=service;migrate-fill-delay=0' ~~~
1d5354ebebac:3000 (172.17.0.2) returned:
ok



## Recluster the cluster 
[Reclustering](https://docs.aerospike.com/docs/reference/info/#recluster) asks the cluster to change its state to accommodate the new quiesces and rebalance data across the new active nodes. Submit the recluster command: ```asadm -e "asinfo -v 'recluster:'"```

```
Admin> asinfo -v 'recluster:'
ip-10-0-151-216.us-east-2.compute.internal:3000 (10.0.151.216) returned:
ignored-by-non-principal

ip-10-0-154-23.us-east-2.compute.internal:3000 (10.0.154.23) returned:
ignored-by-non-principal

ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
ok

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
ignored-by-non-principal

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
ignored-by-non-principal

ip-10-0-19-115.us-east-2.compute.internal:3000 (10.0.19.115) returned:
ignored-by-non-principal
```

It is normal for one node will accept the recluster command.

Run the command locally:

!asadm -e "enable;asinfo -v 'recluster:'"

## Wait for migration to complete

Depending on how much data is stored in the original cluster, there may be ample time to babysit the process by viewing statistics. This allows you to confirm the cluster status and progress. Here are some commands to give visibility and guidance on expected output.

**Note:** This one-node cluster will not have migration status to observe. 

### Confirm the original nodes are quiesced
```asadm -e 'show statistics like quiesce'```

This will provide statistics [```pending_quiesce```](https://docs.aerospike.com/docs/reference/metrics/index.html#pending_quiesce), [```effective_is_quiesced```](https://docs.aerospike.com/docs/reference/metrics/index.html#effective_is_quiesced), and [```nodes_quiesced```](https://docs.aerospike.com/docs/reference/metrics/index.html#nodes_quiesced) for each Namespace. These show which nodes report as being asked to quiesce, which are quiesced, and the number of nodes in the cluster that are quiesced according to each node. 

```
~~~~~~~bar Namespace Statistics (2021-09-15 00:08:31 UTC)~~~~~~~
Node                 |ip-10-0-30-148.us-east-2.compute.internal:3000
effective_is_quiesced|true
nodes_quiesced       |3                                             
pending_quiesce      |true 
Number of rows: 4
```

In [6]:
!asadm -e 'show statistics like quiesce'

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf
~test Namespace Statistics (2021-09-27 22:25:46 UTC)~
Node                 |1d5354ebebac:3000
effective_is_quiesced|false            
nodes_quiesced       |0                
pending_quiesce      |true             
Number of rows: 4



Since this development environment contains only a 1-node cluster, there are no other nodes to accept traffic if this node were to quiesce. This will occur in a multinode cluster if there are no nodes to accept traffic for a namespace.

##### What if one of the new nodes was accidentally quiesced?

If one of the new nodes was accidently quiesced, the configuration can be easily undone.

Undo the quiesce on the 1-node cluster. Ssh to any of the new nodes in the cluster that were accidentally quiesced and execute the following command: ```asinfo -v 'quiesce-undo:'```

In [7]:
!asinfo -v 'quiesce-undo:'

ok


Using asadm 2.2+:

In [8]:
!asadm -e "enable;asinfo -v 'quiesce-undo:' with 172.17.0.2"

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf

~~~ enable ~~~

~~~ asinfo -v 'quiesce-undo:' with 172.17.0.2 ~~~
1d5354ebebac:3000 (172.17.0.2) returned:
ok



### Watch to see when requests stop and a cluster proxying statistics stop increasing
After a few seconds, the quiesced node should no longer receive or proxy data requests. To confirm this, use the following commands and confirm when the statistics stop increasing: ```asadm -e 'show latencies'```

Watch cluster traffic histograms until you are confident that no new requests are sent to quiesced nodes.

```
> asadm -e 'show latencies'
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Latency  (2021-09-15 00:10:15 UTC)~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Namespace|Histogram|                                           Node|ops/sec|>1ms|>8ms|>64ms
test     |write    |ip-10-0-48-227.us-east-2.compute.internal:3000 |    0.0| 0.0| 0.0|  0.0
```

This one-node cluster generally will not have any traffic to view.

### Confirm the original nodes have completed proxying requests:
```
asadm -e 'show statistics like client_proxy'
asadm -e 'show statistics like batch_sub_proxy'
asadm -e 'show statistics like from_proxy_read'
asadm -e 'show statistics like from_proxy_write'
asadm -e 'show statistics like from_proxy_batch_sub'
```

**Note:** This traffic can also include statistics unrelated to adding the Namespace and capacity to the cluster in question. [Configuration of client policies and Namespace mode for strong consistency](https://discuss.aerospike.com/t/understanding-aerospike-server-proxies/3246) can increase these statistics while providing efficient processing of data requests.    

This one-node cluster generally will not be proxying requests.

## Confirm that the cluster is stable with the quiesced nodes
This step is identical to the step titled, “Ensure the cluster is stable”.

Replace ```<Size>``` with the expected current number of cluster nodes and run on one cluster node. 
```asadm -e "asinfo -v 'cluster-stable:size=<Size>;ignore-migrations=no'"```

```
> asadm -e "asinfo -v 'cluster-stable:size=6;ignore-migrations=no'"
ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
206FA51ADD40

ip-10-0-154-23.us-east-2.compute.internal:3000 (10.0.154.23) returned:
206FA51ADD40

ip-10-0-151-216.us-east-2.compute.internal:3000 (10.0.151.216) returned:
206FA51ADD40

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
206FA51ADD40

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
206FA51ADD40

ip-10-0-19-115.us-east-2.compute.internal:3000 (10.0.19.115) returned:
206FA51ADD40
```

Returning the cluster_key (```206FA51ADD40```) indicates that migration is complete.

**Note:** If migrations are not yet complete, the nodes will report ```ERROR::unstable-cluster```. This is the normal state while data is migrating, and is not cause for alarm. Aerospike’s Paxos clustering automatically readjusts for stability and migrates data to ensure resilience independent of cluster configuration and formation. 

```
> asadm -e "asinfo -v 'cluster-stable:size=6;ignore-migrations=no'"
ip-10-0-151-216.us-east-2.compute.internal:3000 (10.0.151.216) returned:
ERROR::unstable-cluster

ip-10-0-48-227.us-east-2.compute.internal:3000 (10.0.48.227) returned:
ERROR::unstable-cluster

ip-10-0-47-149.us-east-2.compute.internal:3000 (10.0.47.149) returned:
ERROR::unstable-cluster

ip-10-0-154-23.us-east-2.compute.internal:3000 (10.0.154.23) returned:
ERROR::unstable-cluster

ip-10-0-19-115.us-east-2.compute.internal:3000 (10.0.19.115) returned:
ERROR::unstable-cluster

ip-10-0-30-148.us-east-2.compute.internal:3000 (10.0.30.148) returned:
ERROR::unstable-cluster
```

# Stop Aerospike on quiesced nodes and remove them from service
Now that the nodes report that migration is complete, it is time to stop Aerospike service on the original cluster nodes and verify that they are removed.


## Stop Aerospike server
To stop Aerospike server, ssh to each original node and use the following sudo command:
```sudo systemctl stop aerospike```

```
$ sudo service aerospike stop
Redirecting to /bin/systemctl stop aerospike.service
```

## Verify the cluster’s node count
To verify that the cluster has 3 nodes, the final node count for this scenario, use the following asadm command:
```asadm -e 'info network'```

```
> asadm -e 'info network'

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Network Information (2021-09-21 23:45:22 UTC)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                           Node|         Node ID|               IP|    Build|Migrations|~~~~~~~~~~~~~~~~~~Cluster~~~~~~~~~~~~~~~~~~|Client|   Uptime
                                               |                |                 |         |          |Size|         Key|Integrity|      Principal| Conns|         
ip-10-0-151-216.us-east-2.compute.internal:3000|*BB9C4FAA0DCD206|10.0.151.216:3000|E-5.5.0.4|   0.000  |   3|B5E3CCE761EB|True     |BB9C4FAA0DCD206|     2|168:28:58
ip-10-0-154-23.us-east-2.compute.internal:3000 | BB91C55C224FC06|10.0.154.23:3000 |E-5.5.0.4|   0.000  |   3|B5E3CCE761EB|True     |BB9C4FAA0DCD206|     2|168:38:02
ip-10-0-19-115.us-east-2.compute.internal:3000 | BB9705CA431B202|10.0.19.115:3000 |E-5.5.0.4|   0.000  |   3|B5E3CCE761EB|True     |BB9C4FAA0DCD206|     2|168:42:55
Number of rows: 3
```

Run this command locally:

In [9]:
!asadm -e 'info network'

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Network Information (2021-09-27 22:25:49 UTC)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
             Node|         Node ID|             IP|     Build|Migrations|~~~~~~~~~~~~~~~~~~Cluster~~~~~~~~~~~~~~~~~~|Client|  Uptime
                 |                |               |          |          |Size|         Key|Integrity|      Principal| Conns|        
1d5354ebebac:3000|*BB9020011AC4202|172.17.0.2:3000|E-5.6.0.13|   0.000  |   1|186B770583C4|True     |BB9020011AC4202|     2|05:14:03
Number of rows: 1



# Clear old nodes from the alumni list
Since the nodes were removed from service, remove them from the cluster’s alumni list to prevent cluster tools from contacting those node IPs in the future. To do so, use the command ```asadm -e "asinfo -v 'services-alumni-reset'"```

```
>asadm -e "asinfo -v 'services-alumni-reset' "
ip-10-0-151-216.us-east-2.compute.internal:3000 (10.0.151.216) returned:
ok

ip-10-0-154-23.us-east-2.compute.internal:3000 (10.0.154.23) returned:
ok

ip-10-0-19-115.us-east-2.compute.internal:3000 (10.0.19.115) returned:
ok
```

Run this command locally:

In [10]:
!asadm -e "enable;asinfo -v 'services-alumni-reset'"

Seed:        [('127.0.0.1', 3000, None)]
Config_file: /home/jovyan/.aerospike/astools.conf, /etc/aerospike/astools.conf

~~~ enable ~~~

~~~ asinfo -v 'services-alumni-reset' ~~~
1d5354ebebac:3000 (172.17.0.2) returned:
ok



# Takeaways
The original 3-node cluster has been replaced with a larger instance 3-node cluster. The new Namespace ```newns``` will be available upon the first node joining the cluster with the Namespace configuration. 

Aerospike clustering adapts to the formation and configuration of a cluster to robustly serve data in realtime. Using AWS Quickstart with CloudFormation, reconfiguring a cluster is efficient and manageable.

## What's Next?

Have questions? Don't hesitate to post about cluster operations on [Aerospike's Discussion Forums](https://discuss.aerospike.com/c/operations/26).

Want to check out other notebooks?
1. [Basic Operations](../python/basic_operations.ipynb)
2. [Java Hello, World](../java/hello_world.ipynb) or [Python Hello, World](../python/hello_world.ipynb)
3. [Python Simple Put Get Example](../python/simple_put_get_example.ipynb)
4. [Java Intro to Data Modeling](../java/java-intro_to_data_modeling.ipynb)
5. [Java Intro to Transactions](../java/java-intro_to_transactions.ipynb)

Are you running this from Binder? [Download the Aerospike Notebook Repo](https://github.com/aerospike-examples/interactive-notebooks) and work with Aerospike Database and Jupyter locally using a Docker container.

## Additional Resources

* Want to get started building with Aerospike? [Try now](https://aerospike.com/lp/try-now/).
* Ready to learn about developing with Aerospike? Go to the [Developer Hub](https://developer.aerospike.com).
* How robust is the Aerospike Database? Browse the [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html).